In [1]:
import pickle
from glob import glob

files = sorted(glob('/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/*.pkl'))
len(files)

9956

In [2]:
from datetime import datetime, timedelta
import pathlib

atleast = datetime(2023, 6, 10)

filtered = []
for f in files:
    fname = pathlib.Path(f)
    if datetime.fromtimestamp(fname.stat().st_mtime) > atleast:
        filtered.append(f)
        
files = sorted(filtered)

In [3]:
from datasets import Audio
from tqdm import tqdm
import numpy as np
import IPython.display as ipd

In [4]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [5]:
with open(files[0], 'rb') as fopen:
    data = pickle.load(fopen)

In [6]:
max(data[0]['asr_model'][0], key=data[0]['asr_model'][0].get)

'ms'

In [7]:
data[0]

{'wav_data': '/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/#AWANIByte_:_Lebuhraya_DASH_-_Laluan_Alternatif_Elakkan_Kesesakkan_[qtWV1Q-7RzU]/0.mp3',
 'start': 0.0,
 'asr_model': ({'ar': 0.0002820058725774288,
   'sq': 1.6484786101500504e-05,
   'tr': 0.0009716476779431105,
   'lv': 1.2892178347101435e-05,
   'pl': 3.6917819670634344e-05,
   'eu': 1.4504192222375423e-05,
   'yi': 5.956028871878516e-06,
   'ru': 0.0002895639627240598,
   'la': 0.010719397105276585,
   'nn': 0.007512533105909824,
   'no': 1.5133452507143375e-05,
   'mg': 3.358373987794039e-07,
   'kk': 1.390142733725952e-05,
   'yo': 0.0011710386024788022,
   'mi': 0.0003330048930365592,
   'fo': 4.8620710003888234e-05,
   'ca': 1.7724236158755957e-06,
   'af': 3.4661305107874796e-05,
   'lt': 8.882307156454772e-06,
   'bo': 0.00010008837125496939,
   'es': 2.7303754905005917e-05,
   'sw': 0.000652390590403229,
   'lo': 0.00010405211651232094,
   'bg': 5.460414831759408e-05,
   'gl': 1.590410761309613e-06,
   '

In [8]:
ms_score = [s['avg_logprob'] for s in data[0]['asr_model'][1]['segments']]
en_score = [s['avg_logprob'] for s in data[0]['asr_model'][2]['segments']]

if np.mean(ms_score) > np.mean(en_score):
    selected = data[0]['asr_model'][1]['segments']
else:
    selected = data[0]['asr_model'][2]['segments']

In [9]:
i = 0

s = selected[i]['start']
e = selected[i]['end']
t = selected[i]['text']
s, e, t

(30.0, 40.0, ' ♪♪')

In [10]:
f = data[0]['wav_data']
audio = reader.decode_example(reader.encode_example(f))['array']
y = audio[int(s * sr): int(e * sr)]

In [11]:
ipd.Audio(y, rate = sr)

In [12]:
import malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [13]:
vad = malaya_speech.vad.webrtc()

In [14]:
y_int = malaya_speech.astype.float_to_int(y)
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr))
np.mean([vad(frame) for frame in frames_int])

0.9760479041916168

In [15]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

In [16]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [17]:
directory = '/home/husein/ssd3/stt/whisper-v5'

In [19]:
# !rm -rf {directory}
!mkdir {directory}

mkdir: cannot create directory ‘/home/husein/ssd3/stt/whisper-v5’: File exists


In [20]:
import torchaudio
import torch
import os

In [21]:
dataset = []
index = 0
for no, f in enumerate(files):
    
    with open(f, 'rb') as fopen:
        data = pickle.load(fopen)
    
    for i in tqdm(range(len(data))):
        lang = max(data[i]['asr_model'][0], key=data[i]['asr_model'][0].get)
        if lang not in {'ms', 'en'}:
            continue
        
        ms_score = [s['avg_logprob'] for s in data[i]['asr_model'][1]['segments']]
        en_score = [s['avg_logprob'] for s in data[i]['asr_model'][2]['segments']]

        if np.mean(ms_score) > np.mean(en_score):
            selected = data[i]['asr_model'][1]['segments']
        else:
            selected = data[i]['asr_model'][2]['segments']
            
        audio = reader.decode_example(reader.encode_example(data[i]['wav_data']))['array']
        for k in range(len(selected)):
            s = selected[k]['start']
            e = selected[k]['end'] + 0.1
            t = selected[k]['text']
            cleaned_t = preprocessing_text(t)
            if not len(cleaned_t):
                continue
                
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = audio[int(s * sr): int(e * sr)]
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
            try:
                y_int = malaya_speech.astype.float_to_int(a)
                frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr, append_ending_trail = False))

                dataset.append(
                    {
                        'file': audio_path,
                        'text': t,
                        'cleaned': cleaned_t,
                        'no': no,
                        'i': i,
                        'k': k,
                        'avg_logprob': selected[k]['avg_logprob'],
                        'no_speech_prob': selected[k]['no_speech_prob'],
                        'vad': float(np.mean([vad(frame) for frame in frames_int])),
                    }
                )
            except:
                pass
            index += 1

100%|███████████████████████████████████████████| 21/21 [00:03<00:00,  5.28it/s]


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  6.26it/s]


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 16.35it/s]


 33%|██████████████▎                            | 27/81 [00:05<00:10,  5.29it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████| 20/20 [00:03<00:00,  5.33it/s]


100%|███████████████████████████████████████████| 13/13 [00:02<00:00,  5.61it/s]


...

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  6.99it/s]


100%|███████████████████████████████████████████| 61/61 [00:09<00:00,  6.35it/s]


100%|█████████████████████████████████████████████| 8/8 [00:01<00:00,  5.68it/s]


100%|███████████████████████████████████████████| 84/84 [00:12<00:00,  6.56it/s]


100%|███████████████████████████████████████████| 13/13 [00:00<00:00, 27.44it/s]


In [24]:
len(dataset)

367026

In [23]:
import json

with open('dataset-whisper-stt-2023-06-19.json', 'w') as fopen:
    json.dump(dataset, fopen)